In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from openai import OpenAI
from key import OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)

In [2]:
# read the responses from the json file
def read_responses(output_file):
    responses_recommendations = []
    with open(output_file, 'r') as f:
        for line in f:
            responses_recommendations.append(eval(line))
    return responses_recommendations

# calculate the hit rate
# hit rate for the recommendations
def hit_rate(recommendation, actual):
    hit = 0
    for rec in recommendation:
        if rec in actual:
            hit += 1
    return hit/len(recommendation)

# NDCG for the recommendations
def ndcg(recommendation, actual):
    dcg = 0
    for i, rec in enumerate(recommendation):
        if rec in actual:
            dcg += 1/np.log2(i+2)
    idcg = 0
    for i in range(len(actual)):
        idcg += 1/np.log2(i+2)
    return dcg/idcg


def eval_response(responses_recommendations, selected_movie_data, original_movie_data, num_users):
    hit_rates = []
    ndcg_scores = []
    for i in range(num_users):
        user_id = selected_movie_data['userId'].unique()[i]
        user_movies = original_movie_data[original_movie_data['userId']==user_id]
        user_watched_movies = user_movies['title'].values
        # print(f"User {user_id} watched movies: {user_watched_movies}")
        recommendation = responses_recommendations[i]
        # print(f"Recommendation: {recommendation}")
        hit_rate_ = hit_rate(recommendation, user_watched_movies)
        # print(f"Hit Rate: {hit_rate_}")
        hit_rates.append(hit_rate_)
        ndcg_ = ndcg(recommendation, user_watched_movies)
        # print(f"NDCG: {ndcg_}")
        ndcg_scores.append(ndcg_)

    avg_hit_rate = np.mean(hit_rates)
    avg_ndcg = np.mean(ndcg_scores)

    print(f"Average Hit Rate: {avg_hit_rate}")
    print(f"Average NDCG: {avg_ndcg}")

    return hit_rates, ndcg_scores

In [3]:
movie_latest_small=pd.read_csv('../../data/processed_movie_latest_small.csv')

# for each user, find his top-most rated movies
N_TOP_MOVIES = 10
def get_top_recent_movies(movies_df, user_id, num_movies=N_TOP_MOVIES):
    user_ratings = movies_df[movies_df['userId'] == user_id]
    user_ratings = user_ratings.sort_values(by=['rating', 'timestamp'], ascending=[False, False])
    return user_ratings.head(num_movies)

# get the top 10 movies for all users
top_movies = pd.DataFrame()
for user_id in movie_latest_small['userId'].unique():
    top_movies = pd.concat([top_movies, get_top_recent_movies(movie_latest_small, user_id, 10)])
top_movies.reset_index(drop=True, inplace=True)

# exclude the top 5 movies from top_movies from movie_latest_small
for user in movie_latest_small['userId'].unique():
    top_movies_user = top_movies[top_movies['userId'] == user]
    top_movies_user_titles = top_movies_user['title'].values
    movie_exclude = movie_latest_small[(movie_latest_small['userId'] == user) & (movie_latest_small['title'].isin(top_movies_user_titles))]
    movie_latest_small_2 = movie_latest_small.drop(movie_exclude.index)
movie_latest_small_2.reset_index(drop=True, inplace=True)

In [4]:
# num_users = 5

# Uncomment below to run on all users
num_users = len(movie_latest_small['userId'].unique())

print(f"Running the recommender on {num_users} users")

Running the recommender on 610 users


In [5]:
avg_hit_rate_basic_list = []
avg_hit_rate_genre_list = []
# avg_hit_rate_summary_list = []
# avg_hit_rate_similar_user_10_20_list = []
# avg_hit_rate_role_list = []
# avg_hit_rate_2step_list = []

avg_ndcg_genre_list = []
avg_ndcg_basic_list = []
# avg_ndcg_summary_list = []
# avg_ndcg_similar_user_10_20_list = []
# avg_ndcg_role_list = []
# avg_ndcg_2step_list = []

for i in range(1,4):
    output_basic_file = f"responses_basic_{i}.json"
    output_genre_file = f"responses_genre_{i}.json"
    # output_summary_file = f"responses_add_summary_{i}.json"
    # output_similar_user_10_20_file = f"responses_similar_user_10_20_{i}.json"
    # output_role_file = f"responses_add_role_{i}.json"
    # output_2step_file = f"responses_2step_{i}.json"

    responses_recommendations_basic = read_responses(output_basic_file)
    responses_recommendations_genre = read_responses(output_genre_file)
    # responses_recommendations_summary = read_responses(output_summary_file)
    # responses_recommendations_similar_user_10_20 = read_responses(output_similar_user_10_20_file)
    # responses_recommendations_role = read_responses(output_role_file)
    # responses_recommendations_2step = read_responses(output_2step_file)

    hit_rate_basic, ndcg_basic = eval_response(responses_recommendations_basic, top_movies, movie_latest_small, num_users)
    avg_hit_rate_basic_list.append(np.mean(hit_rate_basic))
    avg_ndcg_basic_list.append(np.mean(ndcg_basic))

    hit_rate_genre, ndcg_genre = eval_response(responses_recommendations_genre, top_movies, movie_latest_small, num_users)
    avg_hit_rate_genre_list.append(np.mean(hit_rate_genre))
    avg_ndcg_genre_list.append(np.mean(ndcg_genre))

    # hit_rate_summary, ndcg_summary = eval_response(responses_recommendations_summary, top_movies, movie_latest_small, num_users)
    # avg_hit_rate_summary_list.append(np.mean(hit_rate_summary))
    # avg_ndcg_summary_list.append(np.mean(ndcg_summary))

    # hit_rate_similar_user_10_20, ndcg_similar_user_10_20 = eval_response(responses_recommendations_similar_user_10_20, top_movies, movie_latest_small, num_users)
    # avg_hit_rate_similar_user_10_20_list.append(np.mean(hit_rate_similar_user_10_20))
    # avg_ndcg_similar_user_10_20_list.append(np.mean(ndcg_similar_user_10_20))

    # hit_rate_role, ndcg_role = eval_response(responses_recommendations_role, top_movies, movie_latest_small, num_users)
    # avg_hit_rate_role_list.append(np.mean(hit_rate_role))
    # avg_ndcg_role_list.append(np.mean(ndcg_role))

    # hit_rate_2step, ndcg_2step = eval_response(responses_recommendations_2step, top_movies, movie_latest_small, num_users)
    # avg_hit_rate_2step_list.append(np.mean(hit_rate_2step))
    # avg_ndcg_2step_list.append(np.mean(ndcg_2step))

# make a dataframe of the hit rates and ndcg scores
hit_rate_df = pd.DataFrame()
hit_rate_df['avg_hit_rate_basic'] = avg_hit_rate_basic_list
hit_rate_df['avg_hit_rate_genre'] = avg_hit_rate_genre_list
# hit_rate_df['avg_hit_rate_summary'] = avg_hit_rate_summary_list
# hit_rate_df['avg_hit_rate_similar_user_10_20'] = avg_hit_rate_similar_user_10_20_list
# hit_rate_df['avg_hit_rate_role'] = avg_hit_rate_role_list
# hit_rate_df['avg_hit_rate_2step'] = avg_hit_rate_2step_list

ndcg_df = pd.DataFrame()
ndcg_df['avg_ndcg_basic'] = avg_ndcg_basic_list
ndcg_df['avg_ndcg_genre'] = avg_ndcg_genre_list
# ndcg_df['avg_ndcg_summary'] = avg_ndcg_summary_list
# ndcg_df['avg_ndcg_similar_user_10_20'] = avg_ndcg_similar_user_10_20_list
# ndcg_df['avg_ndcg_role'] = avg_ndcg_role_list
# ndcg_df['avg_ndcg_2step'] = avg_ndcg_2step_list

Average Hit Rate: 0.0881967213114754
Average NDCG: 0.019335888231704543
Average Hit Rate: 0.08014903129657228
Average NDCG: 0.01701655085579532
Average Hit Rate: 0.08786885245901639
Average NDCG: 0.019321753738660744
Average Hit Rate: 0.07965722801788376
Average NDCG: 0.017020147810436016
Average Hit Rate: 0.08852459016393442
Average NDCG: 0.01946583698344318
Average Hit Rate: 0.07998509687034278
Average NDCG: 0.017045888423156873


In [6]:
hit_rate_df.describe()

,avg_hit_rate_basic,avg_hit_rate_genre
count,3.000000,3.000000
mean,0.088197,0.079930
std,0.000328,0.000250
min,0.087869,0.079657
25%,0.088033,0.079821
50%,0.088197,0.079985
75%,0.088361,0.080067
max,0.088525,0.080149


In [7]:
ndcg_df.describe()

,avg_ndcg_basic,avg_ndcg_genre
count,3.000000,3.000000
mean,0.019374,0.017028
std,0.000079,0.000016
min,0.019322,0.017017
25%,0.019329,0.017018
50%,0.019336,0.017020
75%,0.019401,0.017033
max,0.019466,0.017046
